In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
p1 = Path.cwd() / 'back_data'
state_fruit = pd.read_csv(p1 / 'state_fruit.csv', index_col=0)
state_fruit

,Apple,Orange,Banana
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


In [3]:
(state_fruit
.stack()
.rename_axis(['state', 'fruit'])
.reset_index(name='weight')
)

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [4]:
state_fruit2 = pd.read_csv(p1 / 'state_fruit2.csv')
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [5]:
state_fruit2.melt(id_vars='State', var_name='Fruit', value_name='Weight')

,State,Fruit,Weight
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [6]:
movie = pd.read_csv(p1 / 'movie.csv')
movie.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [7]:
actor_cols = sorted([col for col in movie.columns if 'actor_' in col or 'movie_title' in col])

In [8]:
actor = movie.loc[:, actor_cols]
actor.head()

,actor_1_facebook_likes,actor_1_name,actor_2_facebook_likes,actor_2_name,actor_3_facebook_likes,actor_3_name,movie_title
0,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,Avatar
1,40000.0,Johnny Depp,5000.0,Orlando Bloom,1000.0,Jack Davenport,Pirates of the Caribbean: At World's End
2,11000.0,Christoph Waltz,393.0,Rory Kinnear,161.0,Stephanie Sigman,Spectre
3,27000.0,Tom Hardy,23000.0,Christian Bale,23000.0,Joseph Gordon-Levitt,The Dark Knight Rises
4,131.0,Doug Walker,12.0,Rob Walker,NaN,NaN,Star Wars: Episode VII - The Force Awakens


In [9]:
def change_col_names(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = (col_name[:5] + col_name[fb_idx - 1:] + col_name[5:fb_idx - 1])
    return col_name

In [10]:
actor2 = actor.rename(change_col_names, axis=1)
actor2.head()

,actor_facebook_likes_1,actor_1,actor_facebook_likes_2,actor_2,actor_facebook_likes_3,actor_3,movie_title
0,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,Avatar
1,40000.0,Johnny Depp,5000.0,Orlando Bloom,1000.0,Jack Davenport,Pirates of the Caribbean: At World's End
2,11000.0,Christoph Waltz,393.0,Rory Kinnear,161.0,Stephanie Sigman,Spectre
3,27000.0,Tom Hardy,23000.0,Christian Bale,23000.0,Joseph Gordon-Levitt,The Dark Knight Rises
4,131.0,Doug Walker,12.0,Rob Walker,NaN,NaN,Star Wars: Episode VII - The Force Awakens


In [11]:
stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = (pd.wide_to_long(actor2, stubnames=stubs, i=['movie_title'], j='actor_num', sep='_')
)
actor2_tidy

,,actor,actor_facebook_likes
movie_title,actor_num,,
Avatar,1,CCH Pounder,1000.0
Pirates of the Caribbean: At World's End,1,Johnny Depp,40000.0
Spectre,1,Christoph Waltz,11000.0
The Dark Knight Rises,1,Tom Hardy,27000.0
Star Wars: Episode VII - The Force Awakens,1,Doug Walker,131.0
...,...,...,...
Signed Sealed Delivered,3,Crystal Lowe,318.0
The Following,3,Sam Underwood,319.0
A Plague So Pleasant,3,David Chandler,0.0


In [12]:
stackme = pd.read_csv(p1 / 'stackme.csv')
stackme2 = stackme.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2', 'd':'group2_a1', 'e':'group2_b2'})
stackme2

,State,Country,group1_a1,group1_b2,Test,group2_a1,group2_b2
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [13]:
pd.wide_to_long(stackme2, stubnames=['group1', 'group2'], i=['State', 'Country', 'Test'], j='Label', sep='_', suffix=r'\w\d')

group1  group2
State Country Test  Label                
TX    US      Test1 a1       0.45       2
                    b2       0.30       6
MA    US      Test2 a1       0.03       9
                    b2       1.20       7
ON    CAN     Test3 a1       0.70       4
                    b2       4.20       2

In [14]:
college = pd.read_csv(p1 / 'college.csv', index_col='INSTNM', usecols=lambda name: 'UGDS_' in name or name == 'INSTNM')
college.head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [15]:
col_stacked = college.stack(dropna=False)
col_stacked

INSTNM                                             
Alabama A & M University                 UGDS_WHITE    0.0333
                                         UGDS_BLACK    0.9353
                                         UGDS_HISP     0.0055
                                         UGDS_ASIAN    0.0019
                                         UGDS_AIAN     0.0024
                                                        ...  
Excel Learning Center-San Antonio South  UGDS_AIAN        NaN
                                         UGDS_NHPI        NaN
                                         UGDS_2MOR        NaN
                                         UGDS_NRA         NaN
                                         UGDS_UNKN        NaN
Length: 67815, dtype: float64

In [16]:
col_stacked.unstack().head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [17]:
col2 = college.reset_index()
col2.head()

,INSTNM,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
0,Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
1,University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
2,Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
3,University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
4,Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [18]:
col_melted = (col2
 .melt(id_vars='INSTNM', var_name='RACE', value_name='PCT')
)
col_melted

,INSTNM,RACE,PCT
0,Alabama A & M University,UGDS_WHITE,0.0333
1,University of Alabama at Birmingham,UGDS_WHITE,0.5922
2,Amridge University,UGDS_WHITE,0.2990
3,University of Alabama in Huntsville,UGDS_WHITE,0.6988
4,Alabama State University,UGDS_WHITE,0.0158
...,...,...,...
67810,SAE Institute of Technology San Francisco,UGDS_UNKN,NaN
67811,Rasmussen College - Overland Park,UGDS_UNKN,NaN
67812,National Personal Training Institute of Cleveland,UGDS_UNKN,NaN
67813,Bay Area Medical Academy - San Jose Satellite ...,UGDS_UNKN,NaN


In [19]:
(col_melted
 .pivot(index='INSTNM', columns='RACE', values='PCT')
 .reindex(col2.loc[:, 'INSTNM'], axis=0)
 .reindex(col2.columns[1:], axis=1)
 .head()
)

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [20]:
emp = pd.read_csv(p1 / 'employee.csv')
emp.head()

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22


In [21]:
(emp
 .groupby(['RACE', 'GENDER'])['BASE_SALARY']
 .mean()
 .astype('int')
 .unstack()
)

GENDER,Female,Male
RACE,,
American Indian or Alaskan Native,60238,60305
Asian/Pacific Islander,63226,61033
Black or African American,48915,51082
Hispanic/Latino,46503,54782
Others,63785,38771
White,66793,63940


In [22]:
def flatten_cols(df):
    df.columns = ['_'.join(col) for col in df.columns.to_flat_index()]
    return df

In [23]:
(emp
 .groupby(['RACE', 'GENDER'])['BASE_SALARY']
 .agg(['mean', 'max', 'min'])
 .astype('int')
 .unstack('GENDER')
 .pipe(flatten_cols)
)

,mean_Female,mean_Male,max_Female,max_Male,min_Female,min_Male
RACE,,,,,,
American Indian or Alaskan Native,60238,60305,98536,81239,26125,26125
Asian/Pacific Islander,63226,61033,130416,163228,26125,27914
Black or African American,48915,51082,150416,275000,24960,26125
Hispanic/Latino,46503,54782,126115,165216,26125,26104
Others,63785,38771,63785,38771,63785,38771
White,66793,63940,178331,210588,27955,26125


In [24]:
flights = pd.read_csv(p1 / 'flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [25]:
fpt = (flights
       .pivot_table(index='AIRLINE', columns='ORG_AIR', values='CANCELLED', aggfunc='sum', fill_value=0)
)
fpt

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0
F9,0,2,1,0,1,1,1,4,0,0
HA,0,0,0,0,0,0,0,0,0,0
MQ,5,0,62,0,0,0,0,85,0,0
NK,1,1,6,0,1,1,3,10,2,0


In [26]:
(flights
 .groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED']
 .sum()
 .unstack(fill_value=0)
)

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0
F9,0,2,1,0,1,1,1,4,0,0
HA,0,0,0,0,0,0,0,0,0,0
MQ,5,0,62,0,0,0,0,85,0,0
NK,1,1,6,0,1,1,3,10,2,0


In [27]:
(flights
 .groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED'])[['DEP_DELAY', 'DIST']]
 .agg(['sum', 'mean'])
 .unstack(['ORG_AIR', 'CANCELLED'], fill_value=0)
 .swaplevel(0, 1, axis=1)
)

sum                                                     \
              DEP_DELAY                                                      
ORG_AIR             ATL          DEN          DFW         IAH          LAS   
CANCELLED             0    1       0    1       0    1      0    1       0   
AIRLINE MONTH                                                                
AA      1         -13.0  0.0   113.0  0.0  4276.0 -3.0  117.0  0.0  1036.0   
        2         -39.0  0.0    71.0  0.0  2662.0  0.0    8.0  0.0   -55.0   
        3          -2.0  0.0    69.0  0.0  5692.0  0.0  109.0  0.0   326.0   
        4           1.0  0.0   304.0  0.0  3518.0  0.0  104.0  0.0   790.0   
        5          52.0  0.0   352.0  0.0  5510.0  0.0   55.0  0.0    93.0   
...                 ...  ...     ...  ...     ...  ...    ...  ...     ...   
WN      7        2604.0  0.0  1919.0  0.0     0.0  0.0    0.0  0.0  4600.0   
        8        1718.0  0.0  1180.0  0.0     0.0  0.0    0.0  0.0  3151.0   
        9        1033.0  0.0   705.0  0.0     0.0  0.0    0.0  0.0  1400.0   
        11        700.0  0.0  1372.0  0.0     0.0  0.0    0.0  0.0  1309.0   
        12       1679.0  0.0  1898.0  0.0     0.0  0.0    0.0  0.0  2327.0   

                    ...         mean                                 \
                    ...         DIST                                  
ORG_AIR             ...          LAX                       MSP        
CANCELLED        1  ...            0            1            0    1   
AIRLINE MONTH       ...                                               
AA      1      0.0  ...  1678.037037  2475.000000   809.000000  0.0   
        2      0.0  ...  1745.892308  1818.000000  1008.000000  0.0   
        3      0.0  ...  1781.567568  1744.000000   964.733333  0.0   
        4      0.0  ...  1850.923913     0.000000   648.714286  0.0   
        5      0.0  ...  1820.478261     0.000000   787.250000  0.0   
...            ...  ...          ...          ...          ...  ...   
WN      7      0.0  ...   912.453704   327.777778   647.266667  0.0   
        8      0.0  ...   835.404040   346.000000   508.703704  0.0   
        9      0.0  ...   830.210000   317.666667   644.416667  0.0   
        11     0.0  ...   748.404040   459.333333   573.642857  0.0   
        12     0.0  ...   811.695238  1067.000000   634.736842  0.0   

                                                                         \
                                                                          
ORG_AIR                ORD                      PHX                 SFO   
CANCELLED                0           1            0      1            0   
AIRLINE MONTH                                                             
AA      1      1068.876033    0.000000  1167.666667    0.0  1860.166667   
        2      1193.782178  771.142857  1311.461538  868.0  1337.916667   
        3      1058.933333  802.000000  1171.363636    0.0  1502.758621   
        4      1094.633094  943.600000  1266.214286    0.0  1646.903226   
        5       998.774775  999.500000  1240.444444    0.0  1436.892857   
...                    ...         ...          ...    ...          ...   
WN      7         0.000000    0.000000   799.160256  369.0   636.210526   
        8         0.000000    0.000000   891.569767    0.0   644.857143   
        9         0.000000    0.000000   872.840000    0.0   731.578947   
        11        0.000000    0.000000   823.258741  872.0   580.875000   
        12        0.000000    0.000000   778.420690    0.0   782.256410   

                       
                       
ORG_AIR                
CANCELLED           1  
AIRLINE MONTH          
AA      1         0.0  
        2      2586.0  
        3         0.0  
        4         0.0  
        5         0.0  
...               ...  
WN      7         0.0  
        8       392.0  
        9       354.5  
        11      392.0  
        12        0.0  

[149 rows x 80 columns]

In [28]:
college = pd.read_csv(p1 / 'college.csv')
college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [29]:
(college
 .groupby(['STABBR', 'RELAFFIL'])[['UGDS', 'SATMTMID']]
 .agg(['size', 'min', 'max'])
 .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis=1)
 .stack('AGG_FUNCS')
 .swaplevel('STABBR', 'AGG_FUNCS', axis=0)
 .sort_index(level='STABBR', ascending=False)
)

AGG_COLS                      UGDS  SATMTMID
AGG_FUNCS RELAFFIL STABBR                   
size      0        WY         11.0      11.0
min       0        WY         52.0     540.0
max       0        WY       9910.0     540.0
size      1        WV          8.0       8.0
          0        WV         65.0      65.0
...                            ...       ...
                   AK          7.0       7.0
min       1        AK         27.0     503.0
          0        AK        109.0       NaN
max       1        AK        275.0     503.0
          0        AK      12865.0       NaN

[332 rows x 2 columns]

In [30]:
weight = pd.read_csv(p1 / 'weightlifting_men.csv')
weight.head()

,Weight Category,M35 35-39,M40 40-44,M45 45-49,M50 50-54,M55 55-59,M60 60-64,M65 65-69,M70 70-74,M75 75-79,M80 80+
0,56,137,130,125,115,102,92,80,67,62,55
1,62,152,145,137,127,112,102,90,75,67,57
2,69,167,160,150,140,125,112,97,82,75,60
3,77,182,172,165,150,135,122,107,90,82,65
4,85,192,182,175,160,142,130,112,95,87,70


In [31]:
weight_melted = (weight
 .melt(id_vars='Weight Category', var_name='Sex-Age', value_name='Total')
)
weight_melted

,Weight Category,Sex-Age,Total
0,56,M35 35-39,137
1,62,M35 35-39,152
2,69,M35 35-39,167
3,77,M35 35-39,182
4,85,M35 35-39,192
...,...,...,...
75,77,M80 80+,65
76,85,M80 80+,70
77,94,M80 80+,75
78,105,M80 80+,80


In [32]:
weight_tidy = (weight_melted.loc[:, 'Sex-Age']
 .str.split(' ', expand=True)
 .rename(dict(enumerate(['Sex', 'Age'])), axis=1)
 .assign(Sex=lambda df: df.loc[:, 'Sex'].str[0])
)
weight_tidy

,Sex,Age
0,M,35-39
1,M,35-39
2,M,35-39
3,M,35-39
4,M,35-39
...,...,...
75,M,80+
76,M,80+
77,M,80+
78,M,80+


In [33]:
(pd.concat([weight_tidy, weight_melted], axis=1)
 .drop('Sex-Age', axis=1)
)

,Sex,Age,Weight Category,Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192
...,...,...,...,...
75,M,80+,77,65
76,M,80+,85,70
77,M,80+,94,75
78,M,80+,105,80


In [35]:
insp = pd.read_csv(p1 / 'restaurant_inspections.csv', parse_dates=['Date'])
insp.head()

,Name,Date,Info,Value
0,E & E Grill House,2017-08-08,Borough,MANHATTAN
1,E & E Grill House,2017-08-08,Cuisine,American
2,E & E Grill House,2017-08-08,Description,Non-food contact surface improperly constructe...
3,E & E Grill House,2017-08-08,Grade,A
4,E & E Grill House,2017-08-08,Score,9.0


In [36]:
(insp
 .pivot(index=['Name', 'Date'], columns='Info', values='Value')
)

,Info,Borough,Cuisine,Description,Grade,Score
Name,Date,,,,,
3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0
...,...,...,...,...,...,...
VALL'S PIZZERIA,2017-03-15,STATEN ISLAND,Pizza/Italian,Wiping cloths soiled or not stored in sanitizi...,A,9.0
VIP GRILL,2017-06-12,BROOKLYN,Jewish/Kosher,Hot food item not held at or above 140Âº F.,A,10.0
WAHIZZA,2017-04-13,MANHATTAN,Pizza,"No facilities available to wash, rinse and san...",A,10.0


In [42]:
(insp
 .set_index(['Name', 'Date', 'Info'])
 .unstack('Info')
 .droplevel(0, axis=1)
 .rename_axis(None, axis=1)
)

,,Borough,Cuisine,Description,Grade,Score
Name,Date,,,,,
3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0
...,...,...,...,...,...,...
VALL'S PIZZERIA,2017-03-15,STATEN ISLAND,Pizza/Italian,Wiping cloths soiled or not stored in sanitizi...,A,9.0
VIP GRILL,2017-06-12,BROOKLYN,Jewish/Kosher,Hot food item not held at or above 140Âº F.,A,10.0
WAHIZZA,2017-04-13,MANHATTAN,Pizza,"No facilities available to wash, rinse and san...",A,10.0


In [43]:
cities = pd.read_csv(p1 / 'texas_cities.csv')
cities

,City,Geolocation
0,Houston,"29.7604° N, 95.3698° W"
1,Dallas,"32.7767° N, 96.7970° W"
2,Austin,"30.2672° N, 97.7431° W"


In [53]:
geos = (cities
 .loc[:, 'Geolocation']
 .str.split(pat=r'°|,', expand=True)
 .rename(dict(enumerate(['latitude', 'lat_direction', 'longitude', 'long_direction'])), axis=1)
 .assign(City=cities.loc[:, 'City'])
)
geos = geos.astype({'latitude':'float', 'longitude':'float'})
geos.dtypes

latitude          float64
lat_direction      object
longitude         float64
long_direction     object
City               object
dtype: object

In [55]:
(cities
 .loc[:, 'Geolocation']
 .str.split(pat=r'°|,', expand=True)
 .rename(dict(enumerate(['latitude', 'lat_direction', 'longitude', 'long_direction'])), axis=1)
 .assign(City=cities.loc[:, 'City'])
 .apply(pd.to_numeric, errors='ignore')
)

,latitude,lat_direction,longitude,long_direction,City
0,29.7604,N,95.3698,W,Houston
1,32.7767,N,96.7970,W,Dallas
2,30.2672,N,97.7431,W,Austin


In [56]:
sensors = pd.read_csv(p1 / 'sensors.csv')
sensors

,Group,Property,2012,2013,2014,2015,2016
0,A,Pressure,928,873,814,973,870
1,A,Temperature,1026,1038,1009,1036,1042
2,A,Flow,819,806,861,882,856
3,B,Pressure,817,877,914,806,942
4,B,Temperature,1008,1041,1009,1002,1013
5,B,Flow,887,899,837,824,873


In [60]:
(sensors
 .melt(id_vars=['Group', 'Property'], var_name='Year', value_name='Number')
 .pivot(index=['Group', 'Year'], columns='Property', values='Number')
 .rename_axis(None, axis=1)
)

Flow  Pressure  Temperature
Group Year                             
A     2012   819       928         1026
      2013   806       873         1038
      2014   861       814         1009
      2015   882       973         1036
      2016   856       870         1042
B     2012   887       817         1008
      2013   899       877         1041
      2014   837       914         1009
      2015   824       806         1002
      2016   873       942         1013

In [72]:
(sensors
 .set_index(['Group', 'Property'])
 .rename_axis('Year', axis=1)
 .stack()
 .unstack('Property')
 .rename_axis(None, axis=1)
)

Flow  Pressure  Temperature
Group Year                             
A     2012   819       928         1026
      2013   806       873         1038
      2014   861       814         1009
      2015   882       973         1036
      2016   856       870         1042
B     2012   887       817         1008
      2013   899       877         1041
      2014   837       914         1009
      2015   824       806         1002
      2016   873       942         1013